## Instanciamento do Spark

In [1]:
# Apontamento dos caminhos da instalação do Spark
import os
import sys

spark_path = "/Users/flavio.clesio/Documents/spark-2.1.0" 

os.environ['SPARK_HOME'] = spark_path
os.environ['HADOOP_HOME'] = spark_path

sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.4-src.zip") # Deve ser o mesmo arquivo da versão do Spark

In [2]:
# Libs para o instanciamento do contexto do Spark e do Spark SQL
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession

In [3]:
# Setup do contexto do Spark
spark = SparkSession \
    .builder \
    .appName("Aula de SQL Spark no Python!") \
    .getOrCreate()

In [4]:
# Configuração do diretório padrão no qual os arquivos estão no disco
ROOT_DIR = "/Users/flavio.clesio/Desktop/"

## Trabalhando com arquivos .txt

In [6]:
# Classe para trabalhar com linhas em um dataframe
from pyspark.sql import Row

In [7]:
# Vamos chamar o contexto do Spark como `sc`
sc = spark.sparkContext

In [8]:
# Agora vamos fazer a carga do arquivo people.txt que está no caminho especificado acima
lines = sc.textFile(ROOT_DIR + "people.txt")

In [10]:
lines.take(5)

[u'Michael, 29', u'Andy, 30', u'Justin, 19']

In [11]:
# No objeto parts vamos gravar via o uso da transformation 'map' todas as palavras do arquivo lines,
# separando por vírgula.
parts = lines.map(lambda l: l.split(","))

In [12]:
parts.collect()

[[u'Michael', u' 29'], [u'Andy', u' 30'], [u'Justin', u' 19']]

In [15]:
# Já no objeto people, vamos realizar um map no RDD parts que foi armazenado acima, e para cada linha vamos realizar
# o apontamento via [chave, valor] chamando a posição 1 de nome e a segunda posição de idade. 
people = parts.map(lambda p: Row(name=p[0], age=int(p[1])))

In [16]:
people.collect()

[Row(age=29, name=u'Michael'),
 Row(age=30, name=u'Andy'),
 Row(age=19, name=u'Justin')]

In [17]:
# Vamos criar um schema chamado 'schemaPeople' em que vamos criar um dataframe com o RDD que foi carregado do txt
# transformado, e armazenado no RDD people
schemaPeople = spark.createDataFrame(people)

In [18]:
schemaPeople.collect()

[Row(age=29, name=u'Michael'),
 Row(age=30, name=u'Andy'),
 Row(age=19, name=u'Justin')]

In [19]:
# Com esse schema instanciado, vamos criar uma vire chamada people
schemaPeople.createOrReplaceTempView("people")

In [22]:
jovens = spark.sql("SELECT * FROM people WHERE age >= 13 AND age <= 19")

In [23]:
jovens.collect()

[Row(age=19, name=u'Justin')]

## Trabalhando com Schemas, Dataframes e RDDs

In [24]:
# Vamos carregar a lib que faz a conversão de data types
from pyspark.sql.types import *

In [25]:
# Vamos instanciar o contexto do Spark mais uma vez
sc = spark.sparkContext

In [26]:
sc

In [27]:
# Carga do arquivo txt
lines = sc.textFile(ROOT_DIR + "/people.txt")

In [28]:
# Quebra das palavras usando o delimitador ','
parts = lines.map(lambda l: l.split(","))

In [29]:
# Nesse caso, a função lambda, realiza a transformação para tupla usando o método strip()
people = parts.map(lambda p: (p[0], p[1].strip()))

In [30]:
people.collect()

[(u'Michael', u'29'), (u'Andy', u'30'), (u'Justin', u'19')]

In [31]:
# Com essa string, vamos colocar o nome das nossas colunas
schemaString = "name age"

In [32]:
# Após isso, vamos declarar os nossos campos de acordo com a divisão do nosso schemaString
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]

In [34]:
fields

[StructField(name,StringType,true), StructField(age,StringType,true)]

In [36]:
# Aqui vamos definir a estrutura do nosso schema
schema = StructType(fields)

In [37]:
schema

StructType(List(StructField(name,StringType,true),StructField(age,StringType,true)))

In [38]:
# Vamos aplicar o Schema no RDD people declarado anteriormente
schemaPeople = spark.createDataFrame(people, schema)

In [40]:
# Vamos aplicar o Schema no RDD people declarado anteriormente
schemaPeople.createOrReplaceTempView("people")

In [45]:
# Como já registramos a nossa o nosso arquivo people em um dataframe, estamos aptos a realizar a consulta nesse objeto
resultado = spark.sql("SELECT name FROM people")

In [43]:
# E no fim temos o nosso resultado
resultado.show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



## Escrita e carga de arquivos Parquet, e consulta em arquivos Parquet.

In [46]:
# Primeiramente vamos realizar a carga do nosso arquivo people.jSON no objeto DF
df = spark.read.load(ROOT_DIR + "people.json", format="json")

In [48]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [49]:
# Se quisermos salvar esse arquivo, podemos selecionar o nome das colunas que queremos
# e passar o formato no qual desejamos que o mesmo seja salvo
df.select("name", "age").write.save(ROOT_DIR + "namesAndAges.parquet", format="parquet")

In [50]:
df = spark.sql("SELECT * FROM parquet.`/Users/flavio.clesio/Desktop/namesAndAges.parquet`")

In [52]:
df.show()

+-------+----+
|   name| age|
+-------+----+
|Michael|null|
|   Andy|  30|
| Justin|  19|
+-------+----+



## Algumas consultas bacanas no Spark ;)

In [53]:
# Carga do arquivo jSON
df = spark.read.json(ROOT_DIR + "people.json")

In [54]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [55]:
# Exibir o Schema e as propriedades dos campos
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [56]:
# Seleção de colunas
df.select("name").show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



In [57]:
# Seleção de colunas
df.select("age").show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [59]:
# Operações aritméticas
df.select(df['name'], df['age']*5).show()

+-------+---------+
|   name|(age * 5)|
+-------+---------+
|Michael|     null|
|   Andy|      150|
| Justin|       95|
+-------+---------+



In [60]:
# Operador condicional
df.filter(df['age'] > 21).show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [61]:
df.groupBy("age").count().show()

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    1|
|  30|    1|
+----+-----+



In [66]:
df.groupBy("age").sum("age").show()

+-------+--------+
|   name|sum(age)|
+-------+--------+
|Michael|    null|
|   Andy|      30|
| Justin|      19|
+-------+--------+



## Exercícios de fixação
###### (Aqui nóis constrói QUERY!)

In [68]:
df = spark.read.json(ROOT_DIR + "brasileiro.json")

In [69]:
df.show(10)

+---+---+---+---+---+---+---+--------------------+---+---+---+------+-----+---+---+---+---+-----+---+
|  %| 1º| 2º| 3º| 4º|  D|  E|             Equipes| GC| GP|  J|Melhor|Média|Pos|Pts|  R| SG|Temp.|  V|
+---+---+---+---+---+---+---+--------------------+---+---+---+------+-----+---+---+---+---+-----+---+
| 56|  3|  1|  3|  2|151|138|           São Paulo|606|845|552|     1|   66|  1|927|  0|239|   14|263|
| 53|  3|  1|  1|  1|182|113|            Cruzeiro|699|870|552|     1|   63|  2|884|  0|171|   14|257|
| 51|  1|  3|  0|  1|174|139|              Santos|681|860|552|     1|   61|  3|856|  0|179|   14|239|
| 51|  0|  3|  1|  0|179|136|       Internacional|632|738|552|     2|   60|  4|849|  1|106|   14|237|
| 52|  3|  0|  1|  1|146|146|         Corinthians|568|697|514|     1|   62|  5|814|  1|129|   13|222|
| 48|  2|  0|  1|  1|189|146|          Fluminense|719|771|552|     1|   57|  6|799|  0| 52|   14|217|
| 47|  1|  0|  2|  1|179|162|            Flamengo|693|722|552|     1|   56|  7|791

In [ ]:
# 1)Quais foram os times que fizeram mais de 220 gols, e que jogaram mais de 10 temporadas?
# 2)Quais foram os top 10 times que mais marcaram gols e que jogaram todos os 14 campeonatos?
# 3)Quais foram os times que nunca ficaram em primeiro, segundo, terceiro e quarto lugares?
# 4)Quais são os clubes que tem o maior saldo de gols negativos (Top 10)?
# 5)Quais foram os times que ganharam o brasileirão mais de 2 vezes?
# 6)Clubes que já foram rebaixados ordenados por quantidade de vezes que foram rebaixados
# 7)Quais foram os times que nunca ficaram em primeiro lugar?
# 8)Quais foram os times ficaram mais vezes em terceiro lugar?
# 9)Quais foram os times que nunca ficaram em terceiro lugar?
# 10)Quais são os clubes que tem a media maior de 60 gols
# 11)Bottom 10 times que menos emparatam
# 12)Quais foram os times ficaram mais vezes em primeiro lugar?
# 13)Bottom 10 em aproveitamento
# 14)Clubes que nunca foram rebaixados
# 15)Tirando os clubes que tem mais de 10 temporadas, quais deles tem o maior número de gols?
# 16)Quais foram os bomttom 10 times que mais marcaram gols e que jogaram todos os 14 campeonatos?
# 17)Quais foram os times que nunca passaram da terceira posição
# 18)Quais foram os times ficaram mais vezes em quarto lugar?
# 19)Top 10 times que mais empataram
# 20)Quais foram os times que nunca ficaram em segundo lugar?
# 21)Quais são os times que tem mais de 50% de aproveitamento?
# 22)Top 10 com melhores aproveitamentos em %
# 23)Quais foram os times ficaram mais vezes em segundo lugar?
# 24)Quais foram os times que nunca ficaram em quarto lugar?